In [ ]:
import os
import time
from PIL import Image
import pytesseract
import re
from datetime import datetime
import mysql.connector

# Tesseract executable path
pytesseract.pytesseract.tesseract_cmd = r'D:\tesseract\tesseract.exe'

# Folder path containing images
folder_path = r"folder path"

# Coordinates for the region to crop (left, upper, right, lower)
specified_region_coordinates = (0, 0, 500, 500)

# Set to keep track of processed files
processed_files = set()

# Database configuration
db_config = {
    'host': 'host name',
    'user': 'username',
    'password': 'password',
    'database': 'db name'
}

# Connect to the database
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Function to extract value from the image
def extract_value_from_image(image):
    cropped_img = image.crop(specified_region_coordinates)
    text = pytesseract.image_to_string(cropped_img)
    matches = re.findall(r'[-+]?\b\d{1,3}(?:,\d{3})*\.\d{2}\b', text)
    if matches:
        pnl_value = matches[0].replace(",", "")  # Remove commas
        return pnl_value
    return None

# Function to get the modified time of the image
def get_image_modified_time(file_path):
    if not os.path.isfile(file_path):
        print(f"File {file_path} does not exist.")
        return None
    modified_time = os.path.getmtime(file_path)
    modified_time_str = datetime.fromtimestamp(modified_time).strftime('%Y-%m-%d %H:%M:%S')
    return modified_time_str

# Function to process new images
def process_new_images():
    global processed_files
    while True:
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.avif')):
                if file_path not in processed_files:
                    img = Image.open(file_path)
                    modified_time = get_image_modified_time(file_path)
                    pnl_value = extract_value_from_image(img)
                    if pnl_value:
                        print(f"File: {filename}, Modified Time: {modified_time}, P&L Value: {pnl_value}")
                        try:
                            insert_query = "INSERT INTO 5paisa (filename, modified_time, pnl_value) VALUES (%s, %s, %s)"
                            cursor.execute(insert_query, (filename, modified_time, pnl_value))
                            conn.commit()
                        except mysql.connector.Error as err:
                            print(f"Error inserting data into MySQL: {err}")
                    processed_files.add(file_path)
        time.sleep(10)

# Start processing images
process_new_images()
